In [931]:
import numpy as np

from numpy.testing import assert_allclose

# Part I. Construct a Householder reflection of a vector.

Given a vector $\mathbf{x}$, and a plane with a normal vector $\mathbf{u}$, the Householder transformation reflects $\mathbf{x}$ about the plane.

The matrix of the Householder transformation is

$$
\mathbf{H} = \mathbf{1} - 2 \mathbf{u} \mathbf{u}^T
$$

Given two equal-length vectors $\mathbf{x}$ and $\mathbf{y}$, a rotation which brings $\mathbf{x}$ to $\mathbf{y}$ is a Householder transform with

$$
\mathbf{u} = \frac{\mathbf{x} - \mathbf{y}}{\left|\mathbf{x} - \mathbf{y}\right|}
$$

Write a function which rotates a given vector, $\mathbf{x} = (x_1, \dots, x_n)$ into $\mathbf{y} = (\left|\mathbf{x}\right|, 0, \dots, 0)^T$ using a Householder transformation.

In [932]:
def householder(vec):
    """Construct a Householder reflection to zero out 2nd and further components of a vector.

    Parameters
    ----------
    vec : array-like of floats, shape (n,)
        Input vector
    
    Returns
    -------
    outvec : array of floats, shape (n,)
        Transformed vector, with ``outvec[1:]==0`` and ``|outvec| == |vec|``
    H : array of floats, shape (n, n)
        Orthogonal matrix of the Householder reflection
    """
    
    vec = np.asarray(vec, dtype=float)
    if vec.ndim != 1:
        raise ValueError("vec.ndim = %s, expected 1" % vec.ndim)
    
    # ... ENTER YOUR CODE HERE ...
    u = np.zeros(len(vec), dtype=float)
    outvec = np.zeros(len(vec), dtype=float)
    outvec[0] = np.sqrt(sum(vec[i]**2 for i in range (len(vec)))) #the first element is the length of vec
    length = np.sqrt(sum((vec[i]-outvec[i])**2 for i in range (len(vec))))  #this is the length of outvec
    u[0] = (sum(-vec[i+1]**2 for i in range(len(vec)-1)))/(vec[0]+np.sqrt(sum(vec[i]**2 for i in range (len(vec)))))/length
# the first element is given by formula (-x_2^2...-x_m^2)/(x1+||x||_2) according to lecture
    for i in range (len(vec)-1):
        u[i+1] = vec[i+1]/length #outvec[i+1]=0 so vec[i+1]-outvec[i+1]=vec[i+1]
    H = np.eye(len(vec)) - 2 * np.multiply(np.repeat(u[:,None],1,axis=1), u) 
# u must be a column, so it has to be transposed as it was given as row vector first
    return outvec, H

Test your function using tests below:

In [933]:
# Test I.1 (10% of the total grade).

v = np.array([1, 2, 3])
v1, h = householder(v)

assert_allclose(np.dot(h, v1), v) 
assert_allclose(np.dot(h, v), v1, atol=1e-7) #the numbers are actually the same, but without atol there is a assert error

In [934]:
# Test I.2 (10% of the total grade).

rndm = np.random.RandomState(1234)

vec = rndm.uniform(size=7)
v1, h = householder(vec)

assert_allclose(np.dot(h, v1), vec)

# Part II. Compute the $\mathrm{QR}$ decomposition of a matrix.

Given a rectangular $m\times n$ matrix $\mathbf{A}$, construct a Householder reflector matrix $\mathbf{H}_1$ which transforms the first column of $\mathbf{A}$ (and call the result $\mathbf{A}^{(1)}$)

$$
\mathbf{H}_1 \mathbf{A} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
&& \dots&& \\
0      & \times & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(1)}\;.
$$

Now consider the lower-right submatrix of $\mathbf{A}^{(1)}$, and construct a Householder reflector which annihilates the second column of $\mathbf{A}$:

$$
\mathbf{H}_2 \mathbf{A}^{(1)} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & 0      & \times & \dots & \times \\
&& \dots&& \\
0      & 0      & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(2)} \;.
$$

Repeating the process $n-1$ times, we obtain

$$
\mathbf{H}_{n-1} \cdots \mathbf{H}_2 \mathbf{H}_1 \mathbf{A} = \mathbf{R} \;,
$$

with $\mathbf{R}$ an upper triangular matrix. Since each $\mathbf{H}_k$ is orthogonal, so is their product. The inverse of an orthogonal matrix is orthogonal. Hence the process generates the $\mathrm{QR}$ decomposition of $\mathbf{A}$. 

Write a function, which receives a recangular matrix, $A$, and returns the Q and R factors of the $QR$ factorization of $A$.

In [935]:
def qr_decomp(a):
    """Compute the QR decomposition of a matrix.
    
    Parameters
    ----------
    a : ndarray, shape(m, n)
        The input matrix
    
    Returns
    -------
    q : ndarray, shape(m, m)
        T)e orthogonal matrix
    r : ndarray, shape(m, n)
        The upper triangular matrix
        
    Examples
    --------
    >>> a = np.random.random(size=(3, 5))
    >>> q, r = qr_decomp(a)
    >>> np.assert_allclose(np.dot(q, r), a)
    
    """
    a1 = np.array(a, copy=True, dtype=float)
    m, n = a1.shape
    
    # ... ENTER YOUR CODE HERE ...
    Q = np.eye(m)
    R = a1
    for i in range (n):
        Q_i = np.eye(m)
        vec = np.zeros(m-i) 
        for l in range(len(vec)): #vector which will be reflected next
            vec[l] = R[i+l,i]
        outvec, H = householder(vec)
        for a in range (H.shape[0]):
            for b in range (H.shape[1]):
                Q_i[a+m-H.shape[0], b+m-H.shape[1]] = H[a,b] #as Q_i must be right upper corner matrix in order
                #matrixes could be multiplied correctly
        Q = Q @ Q_i.T #as in Wikipedia
        R = Q_i @ R
    return Q, R

In [936]:
# Might want to turn this on for prettier printing: zeros instead of `1e-16` etc

np.set_printoptions(suppress=True)

In [937]:
# Test II.1 (20% of the total grade)

rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))
q, r = qr_decomp(a)

# test that Q is indeed orthogonal
assert_allclose(np.dot(q, q.T), np.eye(5), atol=1e-10)

# test the decomposition itself
assert_allclose(np.dot(q, r), a)

Now compare your decompositions to the library function (which actually wraps the corresponding LAPACK functions)

In [938]:
from scipy.linalg import qr
qq, rr = qr(a)

assert_allclose(np.dot(qq, rr), a)
print('qq', "\n", qq, "\n")
print('q', "\n", q, "\n")
print('rr', "\n", rr, "\n")
print('r', "\n", r, "\n")
print('qr', "\n", q @ r, "\n")
print('qq rr', "\n", qq @ rr, "\n")

qq 
 [[-0.13665049  0.53601299  0.09369752  0.661619   -0.49749149]
 [-0.56035895  0.0935397   0.53326881 -0.52477245 -0.34276292]
 [-0.19725922  0.65948912 -0.60068463 -0.37879015  0.14784752]
 [-0.62498418 -0.50418303 -0.52144688  0.18967657 -0.21750907]
 [-0.48765568  0.12171264  0.27224305  0.32774225  0.75222783]] 

q 
 [[ 0.13665049  0.53601299 -0.09369752  0.7697136   0.30459557]
 [ 0.56035895  0.0935397  -0.53326881  0.01839528 -0.62652547]
 [ 0.19725922  0.65948912  0.60068463 -0.32384673 -0.24589462]
 [ 0.62498418 -0.50418303  0.52144688  0.28453698  0.04822969]
 [ 0.48765568  0.12171264 -0.27224305 -0.47049398  0.67223293]] 

rr 
 [[-1.40152769 -1.2514379  -0.89523615]
 [ 0.          0.84158252  0.68447942]
 [ 0.          0.         -0.5496046 ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]] 

r 
 [[ 1.40152769  1.2514379   0.89523615]
 [ 0.          0.84158252  0.68447942]
 [ 0.          0.          0.5496046 ]
 [ 0.         -0.         -0.  

Check if your `q` and `r` agree with `qq` and `rr`. Explain.

# Part III. Avoid forming Householder matrices explicitly.

Note the special structure of the Householder matrices: A reflector $\mathbf{H}$ is completely specified by a reflection vector $\mathbf{u}$. Also note that the computational cost of applying a reflector to a matrix strongly depends on the order of operations:

$$
\left( \mathbf{u} \mathbf{u}^T \right) \mathbf{A}  \qquad \textrm{is } O(m^2 n)\;,
$$
while
$$
\mathbf{u} \left( \mathbf{u}^T \mathbf{A} \right) \qquad \textrm{is } O(mn)
$$

Thus, it seems to make sense to *avoid* forming the $\mathbf{H}$ matrices. Instead, one stores the reflection vectors, $\mathbf{u}$, themselves, and provides a way of multiplying an arbitrary matrix by $\mathbf{Q}^T$, e.g., as a standalone function (or a class).

Write a function which constructs the `QR` decomposition of a matrix *without ever forming the* $\mathbf{H}$ matrices, and returns the $\mathbf{R}$ matrix and reflection vectors. 

Write a second function, which uses reflection vectors to multiply a matrix with $\mathbf{Q}^T$. Make sure to include enough comments for a marker to follow your implementation, and add tests. 

(Peer-graded, 40% of the total grade)

In [939]:
def u_vec(vec): #function which returns u-vector without reflecting vector vec. Mostly copied from Part I
    u = np.zeros(len(vec), dtype=float)
    outvec = np.zeros(len(vec), dtype=float)
    outvec[0] = np.sqrt(sum(vec[i]**2 for i in range (len(vec)))) 
    length = np.sqrt(sum((vec[i]-outvec[i])**2 for i in range (len(vec)))) 
    u[0] = (sum(-vec[i+1]**2 for i in range(len(vec)-1)))/(vec[0]+np.sqrt(sum(vec[i]**2 for i in range (len(vec)))))/length
    for i in range (len(vec)-1):
        u[i+1] = vec[i+1]/length
    return u
    
def R(a): #the first function
    a1 = np.array(a, copy=True, dtype=float)
    m, n = a1.shape
    
    # ... ENTER YOUR CODE HERE ...
    Q = np.eye(m)
    R = a1
    U = np.zeros_like(a) #an array for reflecting vectors
    
    for i in range(n):
        vec = np.zeros(m-i) #chosing vec like in Part II
        for l in range(len(vec)): 
            vec[l] = R[i+l,i]
        u = u_vec(vec) #finding its u-vector
        for j in range (len(u)): #creating reflecting matrix. Note that it is lower triangular in order upcoming
#multiplification to be well-determined
            U[j+i,i] = u[j]
        u_i = np.zeros(m) #vector with first i zero-elements. Again it is necessary because of dimensions
        u_i[i:] = u
        R = R - 2 * np.multiply(np.repeat(u_i[:,None],1,axis=1), u_i @ R) 
#This is the same with H applied to R as H is given as a residue of identity matrix and double scalar product
#of u and u.T. Here I apply u.T @ R first to get scalar and then only u to (u.T @ R) to decrease computational cost. 
#We work with the full R matrix instead of annihilating column by column as done above
    return U, R

def Q(U):
    n, m = U.shape
    Q = np.eye(n)
#here I do literally the same as in R(a) function, considering that R = Q_1*...Q_n @ A. As A=QR, we
#conclude that Q_1*...*Q_n=Q^(-1). Formally I apply Q^(-1) to identity matrix (using u as above, according 
#the formula of Householder reglection) and to get the result the only thing I need is to transpose what I get as 
#matrix Q is orthogonal.
    for i in range(m):
        u = np.zeros(n)
        for j in range(n): 
            u[j] = U[j,i]
        Q = Q - 2 * np.multiply(np.repeat(u[:,None],1,axis=1), u @ Q) 
    return Q.T

u, r = R(a)
q = Q(u)

# test that Q is indeed orthogonal
assert_allclose(np.dot(q, q.T), np.eye(5), atol=1e-10)

# test the decomposition itself
assert_allclose(np.dot(q, r), a)

print('qq', "\n", qq, "\n")
print('q', "\n", q, "\n")
print('rr', "\n", rr, "\n")
print('r', "\n", r, "\n")
print('qr', "\n", q @ r, "\n")
print('qq rr', "\n", qq @ rr, "\n")

#Note that we got exactly the same result as in Part II.

qq 
 [[-0.13665049  0.53601299  0.09369752  0.661619   -0.49749149]
 [-0.56035895  0.0935397   0.53326881 -0.52477245 -0.34276292]
 [-0.19725922  0.65948912 -0.60068463 -0.37879015  0.14784752]
 [-0.62498418 -0.50418303 -0.52144688  0.18967657 -0.21750907]
 [-0.48765568  0.12171264  0.27224305  0.32774225  0.75222783]] 

q 
 [[ 0.13665049  0.53601299 -0.09369752  0.7697136   0.30459557]
 [ 0.56035895  0.0935397  -0.53326881  0.01839528 -0.62652547]
 [ 0.19725922  0.65948912  0.60068463 -0.32384673 -0.24589462]
 [ 0.62498418 -0.50418303  0.52144688  0.28453698  0.04822969]
 [ 0.48765568  0.12171264 -0.27224305 -0.47049398  0.67223293]] 

rr 
 [[-1.40152769 -1.2514379  -0.89523615]
 [ 0.          0.84158252  0.68447942]
 [ 0.          0.         -0.5496046 ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]] 

r 
 [[ 1.40152769  1.2514379   0.89523615]
 [ 0.          0.84158252  0.68447942]
 [-0.          0.          0.5496046 ]
 [ 0.         -0.          0.  